# init
@author: Alexandre Pereira Santos <br>
alexandre.santos(at)lmu.de

## Tasks
- get OpenStreetMap data using the osmnx functionalities in the Overpass API
- get boundary='administrative' and admin_level=10
- clip it to an AOI

## Prerequisites
- have osmnx and geopandas installed
- have an AOI saved as a vector (shapefile or geopackage)

In [ ]:
import osmnx as ox
import geopandas as gpd
from pathlib import Path

# imports

In [ ]:
#import the AOI
AOI_path = Path('../data/processed/')
AOI_file = 'MUM_LIM_BoundingBox_AOI_A.shp'
AOI_gdf = gpd.read_file(AOI_path / AOI_file)

In [ ]:
#bounds provides minx, miny, maxx, maxy
aoi_coords = AOI_gdf.to_crs(epsg='4326').envelope
bbox_df = gpd.GeoDataFrame(geometry=aoi_coords)
print('N',aoi_coords.bounds.values[0][3],'S',aoi_coords.bounds.values[0][1],'W',aoi_coords.bounds.values[0][0],'E',aoi_coords.bounds.values[0][2])

In [ ]:
# get admin polygons for neighbourhoods in the AOI
# List key-value pairs for tags
# - get boundary='administrative' and admin_level=10
tags = {'boundary': ['administrative']} # see https://wiki.openstreetmap.org/wiki/Key:place for more info
col_list = ['osmid','name','geometry', 'admin_level','place'] #OSM has hundreds of cols, we just want a few. For info, check roads.columns'source',,'surface'

places = ox.features_from_bbox(north=aoi_coords.bounds.values[0][3],
                              south=aoi_coords.bounds.values[0][1],
                              west=aoi_coords.bounds.values[0][0],
                              east=aoi_coords.bounds.values[0][2],
                              tags=tags)
places.reset_index(inplace=True)

#filtering out the wrong geometries (POLYGON or POINT) and element types (NODE)
places = places.loc[(places.geometry.type=='Polygon')&(places.admin_level.notna()), col_list] #&(places.element_type == 'way') 
#convert the CRS of roads to the same as AOI
places.to_crs(AOI_gdf.crs,inplace=True)

#plot them
places.plot('name',ax=AOI_gdf.plot(figsize=(10, 10),  alpha=0.5, color='lightgray'),cmap='Paired', zorder=2) #linewidth=1,

In [ ]:
places['admin_level'].sort_values().unique()

In [ ]:
places

In [ ]:
places.columns
#.to_file('../data/processed/LIM_rs_floods_2024_guaiba_neighbourhoods_A.shp')

In [ ]:
places.type.unique()

# export

In [ ]:
#export places to a shapefile
export_path = Path('../data/processed/')
places.to_file(AOI_path / 'MUM_LIM_MMR_municipalities_A.shp')